In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
        
def flatten(list_1):
    list_2 = []
    for each in list(list_1):
        try:
            for each_2 in each:
                list_2.append(each_2)
        except:
            pass
    return list_2
        
# ~
#Default Ending

import os
from tqdm import tqdm
import random

print('OK !')

OK !


In [2]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor, TfidfRetriever
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline

from flask import Flask, request, jsonify

print('OK !')

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


OK !


# Embedding Retrieval

In [3]:
# For EmbeddingRetriever
similarity_type = "cosine"

container_name = "es1"

er_100 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_100",
                                           similarity=similarity_type, embedding_dim=768)
er_200 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_200",
                                           similarity=similarity_type, embedding_dim=768)
er_300 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_300",
                                           similarity=similarity_type, embedding_dim=768)

# Dense Passage Retrieval

In [4]:
# For DensePassageRetriever
similarity_type = "dot_product"

dpr_100 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_dpr_100",
                                           similarity=similarity_type)
dpr_200 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_dpr_200",
                                           similarity=similarity_type)
dpr_300 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_dpr_300",
                                           similarity=similarity_type)

# Set document_store 

In [5]:
document_store_list = [er_200, dpr_200]

# Set Retriever

In [6]:
tfidf_retriever = TfidfRetriever(document_store_list[0])

INFO - backoff -  Backing off send_request(...) for 0.2s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - backoff -  Backing off send_request(...) for 0.2s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - haystack.nodes.retriever.sparse -  Found 18455 candidate paragraphs from 18455 docs in DB
INFO - backoff -  Backing off send_request(...) for 3.9s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(

In [7]:
dpr_retriever = DensePassageRetriever(
    document_store=document_store_list[1],
    query_embedding_model="../shared_data/thanatcc/RIA_Model/DPR-retriever/query_encoder",
    passage_embedding_model="../shared_data/thanatcc/RIA_Model/DPR-retriever/passage_encoder",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1
INFO - haystack.modeling.utils -  Number of GPUs: 2
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../shared_data/thanatcc/RIA_Model/DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  Loaded ../shared_data/thanatcc/RIA_Model/DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../shared_data/thanatcc/RIA_Model/DPR-retriever/passage_encoder
ERROR - backoff -  Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CE

In [8]:
er_retriever = EmbeddingRetriever(
   document_store=document_store_list[0],
   embedding_model="../shared_data/thanatcc/RIA_Model/all-mpnet-base-v1",
   model_format="sentence_transformers",
   use_gpu = True,
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1
INFO - haystack.modeling.utils -  Number of GPUs: 2
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model ../shared_data/thanatcc/RIA_Model/all-mpnet-base-v1


# Set Reader

In [9]:
# reader_path = "../shared_data/thanatcc/RIA_Model/albert_xxlargev1_squad2_512-reader"
reader_path = "../shared_data/thanatcc/RIA_Model/minilm-uncased-squad2-reader"

In [10]:
reader_1 = FARMReader(model_name_or_path=reader_path, 
                      use_gpu=1)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../shared_data/thanatcc/RIA_Model/minilm-uncased-squad2-reader
INFO - backoff -  Backing off send_request(...) for 1.0s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - haystack.modeling.model.language_model -  Loaded ../shared_data/thanatcc/RIA_Model/minilm-uncased-squad2-reader
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused paramet

In [11]:
reader_1.__dict__['use_gpu']

1

In [12]:
reader_2 = FARMReader(model_name_or_path=reader_path, 
                      use_gpu=2)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../shared_data/thanatcc/RIA_Model/minilm-uncased-squad2-reader
ERROR - backoff -  Giving up send_request(...) after 4 tries (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
ERROR - posthog -  error uploading: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate 

In [13]:
reader_2.__dict__['use_gpu']

2

In [14]:
reader_list = [reader_1, reader_2]
reader_list

random.choice(reader_list)

# Set Meta Filter Dict

In [15]:
%%time

meta_list = list(document_store_list[0])
meta_list = [each.__dict__['meta'] for each in meta_list]
meta_df = pd.DataFrame(meta_list)

# By Sentence
main_col_list = ['central_bank', 'keyword']

# Editing this will depend on NER models
alt_col_list = ['ORG', 'LOC', 'PER', 'MISC']

fil_col_list = main_col_list + alt_col_list
min_count = 3
meta_fil_dict = {}
for col in fil_col_list:
    if col in alt_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(flatten(list(meta_df[col].values)))[0].value_counts().reset_index()
    elif col in main_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(list(meta_df[col].values))[0].value_counts().reset_index()
    
    if col in list(meta_df.columns):
        temp_df = temp_df[temp_df[0] >= min_count]
        temp_dict = dict(zip(temp_df['index'], temp_df[0]))
        meta_fil_dict[col] = temp_dict
        
return_meta_fil_dict = {}
return_meta_fil_dict['sentence'] = meta_fil_dict

# By Document files
meta_df = meta_df.groupby('file_name').first().reset_index()
min_count = 0
fil_col_list = ['central_bank', 'keyword']
meta_fil_dict = {}
for col in fil_col_list:
    if col in alt_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(flatten(list(meta_df[col].values)))[0].value_counts().reset_index()
    elif col in main_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(list(meta_df[col].values))[0].value_counts().reset_index()
    
    if col in list(meta_df.columns):
        temp_df = temp_df[temp_df[0] >= min_count]
        temp_dict = dict(zip(temp_df['index'], temp_df[0]))
        meta_fil_dict[col] = temp_dict
        
return_meta_fil_dict['document'] = meta_fil_dict

INFO - backoff -  Backing off send_request(...) for 0.0s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - backoff -  Backing off send_request(...) for 0.6s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))))
INFO - backoff -  Backing off send_request(...) for 0.9s (requests.exceptions.SSLError: HTTPSConnectionPool(host='tm.hs.deepset.ai', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable t

CPU times: user 16.5 s, sys: 1.23 s, total: 17.7 s
Wall time: 27.3 s


In [16]:
# retriever_pipe = DocumentSearchPipeline(retriever)
# reader_pipe = ExtractiveQAPipeline(reader,retriever)

# Set API Server

In [17]:
app = Flask(__name__)

@app.route('/haystack_200_reader_pipe', methods=['POST'])
def haystack_200_reader_pipe():
    query_list = []
    if request.method == 'POST':
        body = request.get_json()
        if body['retriever_type'] == 'Exact Match':
            reader_pipe = ExtractiveQAPipeline(random.choice(reader_list), tfidf_retriever)
        elif body['retriever_type'] == 'Semantic Search (DPR)':
            reader_pipe = ExtractiveQAPipeline(random.choice(reader_list), dpr_retriever)
        elif body['retriever_type'] == 'Semantic Search (ER)':
            reader_pipe = ExtractiveQAPipeline(random.choice(reader_list), er_retriever)
            
        prediction = reader_pipe.run(
            query=body['query'], params={"Retriever": {"top_k": int(body['top_k_retriever'])},
                                         "Reader": {'top_k': int(body['top_k_reader'])},
                                         "filters": body['filters'],
                                         }
        )
        ans_dict = prediction['answers']
        ans_dict = [each.__dict__ for each in ans_dict]
        prediction['answers'] = ans_dict
        doc_dict = prediction['documents']
        doc_dict = [each.__dict__ for each in doc_dict]
        prediction['documents'] = doc_dict
        
        return prediction, 201
    
@app.route('/haystack_200_retriever_pipe', methods=['POST'])
def haystack_200_retriever_pipe():
    query_list = []
    if request.method == 'POST':
        body = request.get_json()
        if body['retriever_type'] == 'Exact Match':
            retriever_pipe = DocumentSearchPipeline(tfidf_retriever)
        elif body['retriever_type'] == 'Semantic Search (DPR)':
            retriever_pipe = DocumentSearchPipeline(dpr_retriever)
        elif body['retriever_type'] == 'Semantic Search (ER)':
            retriever_pipe = DocumentSearchPipeline(er_retriever)
        prediction = retriever_pipe.run(
            query=body['query'], params={"Retriever": {"top_k": int(body['top_k_retriever'])},
                                         "filters": body['filters'],
                                         }
        )
        
        doc_list = [each.__dict__ for each in prediction['documents']]
        prediction['documents'] = doc_list
        return prediction, 201
    
@app.route('/haystack_200_meta', methods=['GET'])
def haystack_200_meta():
    query_list = []
    if request.method == 'GET':        
        return return_meta_fil_dict, 201
    
print('OK !')

OK !


In [ ]:
app.run(port = 5201)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO - werkzeug -   * Running on http://127.0.0.1:5201 (Press CTRL+C to quit)
